# Generate Color Bins For Each Layer

### 0. Import libraries and create empty dictionaries

In [29]:
import geopandas as gpd
import numpy as np
import mapclassify as mc
import matplotlib.pyplot as plt

In [30]:
total_dict = {}
dph_illinois_dict = {}
vul_dict = {}

In [31]:
dynamic_dict = {}
dynamic_illinois_dict = {}

In [32]:
def create_dict(column):
    tmp_dict = {}
    tmp_dict['Quantiles'] = {
        'bins': ','.join(mc.Quantiles(column, k=6).bins.astype(str)),
        'k': 6
    }
    tmp_dict['FisherJenks'] = {
        'bins': ','.join(mc.FisherJenks(column, k=6).bins.astype(str)),
        'k': 6
    }
    tmp_dict['NaturalBreaks'] = {
        'bins': ','.join(mc.NaturalBreaks(column, k=6).bins.astype(str)),
        'k': 6
    }
    return tmp_dict

### 3. For IDPH County-level Data

#### 3.0 Load Data

In [33]:
dph_illinois_gdf = gpd.read_file('./dph_county_data.geojson')

In [34]:
dph_illinois_gdf.head(5)

,id,population,NAME,cases_ts,dt_first_case,today_case,today_new_case,deaths_ts,dt_first_death,today_death,today_new_death,today_tested,today_new_tested,start,end,dt_unit,change_ts,geometry
0,Chicago,2693976,Chicago,"0,0,0,0,0,519,519,782,915,1161,1364,1610,2026,...",2020-03-22,330697,396,"0,0,0,0,0,4,4,5,5,9,9,12,16,16,21,39,41,57,76,...",2020-03-22,5920,0,7013885,25958,2020-03-17,2021-11-07,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,2.24,1.44,1.37,0.82,...","MULTIPOLYGON (((-87.93514 42.00089, -87.93521 ..."
1,McHenry,308570,McHenry,"2,4,6,8,11,12,12,14,19,27,45,47,52,52,63,69,81...",2020-03-17,35984,71,"0,0,0,0,0,0,0,0,0,1,1,2,2,2,2,2,3,3,3,3,3,4,5,...",2020-03-26,336,0,569967,2559,2020-03-17,2021-11-07,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,3.08,2.33,1.57,0...","POLYGON ((-88.70742 42.49352, -88.70741 42.493..."
2,Boone,53577,Boone,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,4,6,7,...",2020-04-05,8119,15,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...",2020-04-08,84,0,93429,255,2020-03-17,2021-11-07,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,4.0,...","POLYGON ((-88.70742 42.49352, -88.70750 42.493..."
3,Ogle,50923,Ogle,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,4,5,7,10,1...",2020-03-31,7840,15,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-04-15,94,0,96607,304,2020-03-17,2021-11-07,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,1.0,4....","POLYGON ((-89.68809 42.19950, -89.68807 42.184..."
4,Will,692310,Will,"2,3,9,10,12,21,21,28,40,67,104,127,182,223,228...",2020-03-17,93764,154,"0,0,0,1,1,1,1,1,2,3,3,4,4,5,8,9,8,10,11,12,19,...",2020-03-20,1144,1,1545447,5793,2020-03-17,2021-11-07,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,9.63,6.69,6.62,4.47,...","POLYGON ((-88.26146 41.72439, -88.26103 41.708..."


In [35]:
dph_illinois_gdf = dph_illinois_gdf[dph_illinois_gdf['id'] != 'Illinois']

#### 3.1 Get Constant Classes from today's data

In [36]:
illinois_case = dph_illinois_gdf['today_case']
illinois_death = dph_illinois_gdf['today_death']
illinois_case_per_100k_capita = dph_illinois_gdf['today_case']/(dph_illinois_gdf['population']/100000)
illinois_death_per_100k_capita = dph_illinois_gdf['today_death']/(dph_illinois_gdf['population']/100000)
#death_case_ratio = counties_gdf['today_death']/counties_gdf['today_case']
#death_case_ratio = death_case_ratio.replace(np.nan, 0)

In [37]:
illinois_death_per_100k_capita = illinois_death_per_100k_capita.replace(np.nan,0)
illinois_death_per_100k_capita = illinois_death_per_100k_capita.replace(np.inf,0)
illinois_case_per_100k_capita = illinois_case_per_100k_capita.replace(np.nan,0)
illinois_case_per_100k_capita = illinois_case_per_100k_capita.replace(np.inf,0)

In [38]:
illinois_log_case = illinois_case.apply(lambda x: np.log(x + 1))
illinois_log_death = illinois_death.apply(lambda x: np.log(x + 1))
illinois_log_case_per_100k_capita = illinois_case_per_100k_capita.apply(lambda x: np.log(x + 1))
illinois_log_death_per_100k_capita =illinois_death_per_100k_capita.apply(lambda x: np.log(x + 1))

In [39]:
dph_illinois_dict['case'] = {}
dph_illinois_dict['death'] = {}
dph_illinois_dict['case_per_100k_capita'] = {}
dph_illinois_dict['death_per_100k_capita'] = {}

In [40]:
dph_illinois_dict['case']['nolog'] = create_dict(illinois_case)
dph_illinois_dict['case']['log'] = create_dict(illinois_log_case)
dph_illinois_dict['death']['nolog'] = create_dict(illinois_death)
dph_illinois_dict['death']['log'] = create_dict(illinois_log_death)
dph_illinois_dict['case_per_100k_capita']['nolog'] = create_dict(illinois_case_per_100k_capita)
dph_illinois_dict['case_per_100k_capita']['log'] = create_dict(illinois_log_case_per_100k_capita)
dph_illinois_dict['death_per_100k_capita']['nolog'] = create_dict(illinois_death_per_100k_capita)
dph_illinois_dict['death_per_100k_capita']['log'] = create_dict(illinois_log_death_per_100k_capita)

In [41]:
dph_illinois_dict

{'case': {'nolog': {'Quantiles': {'bins': '1952.0000000000005,2696.0,4418.0,7103.000000000002,16700.000000000025,330697.0',
    'k': 6},
   'FisherJenks': {'bins': '8501,23620,42956,82227,111403,330697', 'k': 6},
   'NaturalBreaks': {'bins': '8501.0,27789.0,42956.0,82227.0,111403.0,330697.0',
    'k': 6}},
  'log': {'Quantiles': {'bins': '7.57712193087668,7.89989532313973,8.39366870513074,8.868413284672005,9.72322387685162,12.708960851174155',
    'k': 6},
   'FisherJenks': {'bins': '7.273786317844895,8.24800570160062,9.048056708918736,10.23243152273655,11.620918512473422,12.708960851174155',
    'k': 6},
   'NaturalBreaks': {'bins': '7.273786317844895,8.24800570160062,9.048056708918736,10.23243152273655,11.448546931034022,12.708960851174155',
    'k': 6}}},
 'death': {'nolog': {'Quantiles': {'bins': '23.000000000000014,40.000000000000014,71.0,102.00000000000001,226.00000000000037,5920.0',
    'k': 6},
   'FisherJenks': {'bins': '157,376,891,1392,5413,5920', 'k': 6},
   'NaturalBreaks'

### 7. Summary and ouput

In [42]:
import json
total_dict['dph_illinois'] = dph_illinois_dict
with open('classes_idph.json','w') as json_file:
    json.dump(total_dict, json_file)
print('done')

done
